In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN_light_cpd_64"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.wavelet_conv_model_light as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp7.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [5]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [6]:
train_data.shape, train_gt.shape

((1358, 1024, 77), (1358, 1024, 1))

In [7]:
test_data.shape, test_gt.shape

((233, 1024, 77), (233, 1024, 1))

## Model

In [8]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

wavelet_levels = 5    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_kernel_size = 5    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)


In [9]:
train_data.shape

(1358, 1024, 77)

In [10]:
pos_weight.shape

(1,)

In [11]:
pos_weight

array([ 2.15172601], dtype=float32)

In [12]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/checkpoint' 

In [14]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [15]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_kernel_size = wavelet_kernel_size,    
                                 activation = activation,
                                 trainable_wavelet=False,
                                 trainable_conv=False,
                                 trainable_last=True)

In [16]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'change')

In [17]:
#load the model
saver = tf.train.Saver(var_list = [
    model.weights['wc0'],model.weights['wc1'],model.weights['wc2'],
    model.bias['bc0'],model.bias['bc1'],model.bias['bc2'],
    model.weights['high_pass'],model.weights['low_pass']
])
saver.restore(sess, '../model/DWN9009_opp/checkpoint-4000')

INFO:tensorflow:Restoring parameters from ../model/DWN9009_opp/checkpoint-4000


In [18]:
trainer.run_debug_ops()

{'input_shape': array([ 100, 1024,   77], dtype=int32)}


In [19]:
trainer.train(max_iter = 20000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Train Batch Evaluation
AUC = 0.300 vs. 0.280
F1 = 0.431 vs. 0.431
Precision = 0.275 vs. 0.275
Recall = 0.998 vs. 1.000
Validation
AUC = 0.268 vs. 0.188
F1 = 0.325 vs. 0.317
Precision = 0.246 vs. 0.189
Recall = 0.477 vs. 0.999
Minibatch Loss= 5.230678
Train Batch Evaluation
AUC = 0.525 vs. 0.287
F1 = 0.526 vs. 0.464
Precision = 0.467 vs. 0.302
Recall = 0.602 vs. 1.000
Validation
AUC = 0.463 vs. 0.194
F1 = 0.497 vs. 0.317
Precision = 0.505 vs. 0.189
Recall = 0.489 vs. 0.997
Minibatch Loss= 2.799953
Train Batch Evaluation
AUC = 0.602 vs. 0.291
F1 = 0.567 vs. 0.422
Precision = 0.454 vs. 0.271
Recall = 0.756 vs. 0.955
Validation
AUC = 0.474 vs. 0.184
F1 = 0.513 vs. 0.317
Precision = 0.495 vs. 0.188
Recall = 0.531 vs. 1.000
Minibatch Loss= 1.637616
Train Batch Evaluation
AUC = 0.640 vs. 0.360
F1 = 0.645 vs. 0.511
Precision = 0.528 vs. 0.343
Recall = 0.830 vs. 1.000
Validation
AUC = 0.494 vs. 0.205
F1 = 0.521 vs. 0.318
Precision = 0.455 vs. 0.189
Recall = 0.610 vs. 0.993
Minibatch Loss= 1.630

In [20]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

'../model/DWN_light_cpd_64/checkpoint-20000'

In [ ]:
#load the model
saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [21]:
#This is the model evaluation on test dataset
result = nn_eval.evaluate_model (model, sess, test_data, test_gt)

AUC = 0.653 vs. 0.184
F1 = 0.637 vs. 0.318
Precision = 0.568 vs. 0.189
Recall = 0.725 vs. 0.997


In [22]:
probability_of_pos = nn_eval.deploy(model, sess, test_data, test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})

In [ ]:
1